In [1]:
# Imports
import sys
from pathlib import Path
from typing import Optional
import time


def find_project_root(
    start: Path = Path.cwd(), marker: str = "pyproject.toml"
) -> Optional[Path]:
    for p in [start] + list(start.parents):
        if (p / marker).is_file():
            return p
    return None


PROJECT_ROOT: Optional[Path] = find_project_root()
if PROJECT_ROOT:
    proj_path = str(PROJECT_ROOT)
    if proj_path not in sys.path:
        sys.path.insert(0, proj_path)
else:
    print(
        f"Warning: pyproject.toml not found in parents of {Path.cwd()}; sys.path unchanged"
    )

In [2]:
from src.servo.classes_servo import ServoController

In [3]:
# Create the servo controller for testing/demo
servo_controller: ServoController = ServoController(channel=0)

PCA9685 initialized successfully


In [4]:
target_angle: float = 0.0
speed_deg_per_sec: float = 30.0  # degrees per second

In [7]:
servo_controller.move_to_center(speed_deg_per_sec=30.0)

In [ ]:
# for current_counts in range(307, 0, -1):

#     print(f"Setting servo to counts: {current_counts}")
#     servo_controller._drv_pwm.set_pwm(
#         0, 0, current_counts
#     )

#     time.sleep(0.5)

In [6]:
servo_controller.move_by_angle(-60.0, speed_deg_per_sec=30.0)

In [ ]:
for i in range(5):
    servo_controller.move_to(target_angle, speed_deg_per_sec=speed_deg_per_sec)
    servo_controller.move_to_center(speed_deg_per_sec=speed_deg_per_sec)

In [ ]:
servo_controller.move_to(0.0, speed_deg_per_sec=30.0)